In [1]:
import pandas as pd
import pandas_gbq
import numpy as np
import yaml
from google.cloud import bigquery
from box import ConfigBox
from datetime import datetime, timedelta

client = bigquery.Client()

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
scrub_date_cutoff = '2025-02-06'

In [4]:
abcl_base = pd.read_csv('ABCL_base.csv')
abcl_base.info()
print()
cust_base = pd.read_csv('app_custs.csv')
cust_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125014 entries, 0 to 125013
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   customer_id     125014 non-null  int64 
 1   Disbursal_date  125014 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.9+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771998 entries, 0 to 6771997
Data columns (total 2 columns):
 #   Column        Dtype
---  ------        -----
 0   customer_id   int64
 1   mobilenumber  int64
dtypes: int64(2)
memory usage: 103.3 MB


In [6]:
cust_base.shape[0] == cust_base.customer_id.nunique()

True

In [7]:
grouped = cust_base.groupby(['mobilenumber']).agg(
    row_count = ('customer_id', 'count')
)

problem_mobilenums = grouped[grouped.row_count > 1].reset_index().mobilenumber.to_list()

In [8]:
clean_base = cust_base[~cust_base.mobilenumber.isin(problem_mobilenums)].reset_index(drop=True)
clean_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6771972 entries, 0 to 6771971
Data columns (total 2 columns):
 #   Column        Dtype
---  ------        -----
 0   customer_id   int64
 1   mobilenumber  int64
dtypes: int64(2)
memory usage: 103.3 MB


In [9]:
final_base = pd.merge(
    abcl_base,
    clean_base,
    how='inner',
    on='customer_id'
)

In [10]:
grouped = final_base.groupby(['mobilenumber', 'Disbursal_date']).agg(
    row_count = ('mobilenumber', 'count')
)

grouped[grouped.row_count > 1].shape

(35, 1)

In [26]:
final_base.mobilenumber = final_base.mobilenumber.astype(str)

In [27]:
pandas_gbq.to_gbq(
    dataframe=final_base.rename(
        columns={'Disbursal_date': 'disbursal_date'}
    )[['mobilenumber', 'disbursal_date']].drop_duplicates(),
    destination_table='abcd_data_science_app.abcl_shared_base_cleaned',
    project_id='abcd-dataplatform',
    if_exists='replace'
)

100%|██████████| 1/1 [00:00<00:00, 9425.40it/s]


In [12]:
pandas_gbq.to_gbq(
    dataframe=abcl_base,
    destination_table='abcd_data_science_app.abcl_shared_base',
    project_id='abcd-dataplatform',
    if_exists='replace'
)

100%|██████████| 1/1 [00:00<00:00, 3795.75it/s]


# Automated Section

In [2]:
month_map = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec',    
}

## Customers Registered in the specified month-year (Base)

In [3]:
# app_month_registration.customer_id.nunique()

## Extract DPD information from Experian

### Selecting columns and Safe Casting to approriate data types from String

In [4]:
date_columns = [
    'balance_dt',
    'open_dt',
    'closed_dt'
]

casted_date_columns = [
    f"SAFE.PARSE_DATE('%d/%m/%Y', {col}) AS {col}"
    for col in date_columns
]

dpds = 25
dpd_columns = [f"days_past_due_{str(i).zfill(2)}" for i in range(1, dpds+1)]
casted_dpd_columns = [f"CASE WHEN SAFE_CAST({col} AS FLOAT64) != -1 THEN SAFE_CAST({col} AS FLOAT64) END AS {col}" for col in dpd_columns]

amount_columns = [
    'balance_am',
    'orig_loan_am',
    'credit_limit_am'
]

casted_amount_columns = [
    f"SAFE_CAST({col} AS FLOAT64) AS {col}"
    for col in amount_columns
]

column_list = [
    "CUSTOMER_ID AS mobilenumber", 
    "acct_key",
    "acct_type_cd", 
    'm_sub_id',
    "scrub_date"
]

column_list.extend(casted_date_columns)
column_list.extend(casted_amount_columns)
column_list.extend(casted_dpd_columns)

In [5]:
updated_column_list = [
    "exp.mobilenumber", 
    "acct_key",
    "acct_type_cd", 
    "scrub_date"
]

updated_column_list.extend(date_columns)
updated_column_list.extend(amount_columns)
updated_column_list.extend(dpd_columns)

### Fix issues with NaN values in dpd columns (only applied when there are values in the previous and next DPD column)

In [6]:
query_parts = []

for i in range(len(dpd_columns)):
    current = dpd_columns[i]
    
    if i == 0:
        # Skip for the first column (no dpd_{n-1})
        query_parts.append(f"exp.{current}")
    elif i == len(dpd_columns) - 1:
        # Skip for the last column (no dpd_{n+1})
        pass
    else:
        prev_col = dpd_columns[i - 1]
        next_col = dpd_columns[i + 1]
        query_parts.append(
            f"""
        CASE 
            WHEN (exp.{current} IS NULL OR exp.{current} <= 0)                                -- Check if DPD_{str(i+1).zfill(2)} is NULL, -1, or 900.
                 AND exp.{prev_col} IS NOT NULL                                                       -- Check if DPD_{str(i).zfill(2)} is NOT NULL
                 AND exp.{next_col} IS NOT NULL                                                       -- Check if DPD_{str(i+2).zfill(2)} is NOT NULL
                 AND exp.{prev_col} - exp.{next_col}> 55                                        -- Check if DPD_{str(i+2).zfill(2)} - DPD_{str(i).zfill(2)} > 55 
            THEN exp.{next_col} + 30                                                                  -- If all above conditions satisfy, add 30 to DPD_{str(i).zfill(2)} and store as fixed value
            ELSE exp.{current} 
        END AS {current}"""
        )

dpd_nan_fix = ', '.join(query_parts)

### Calculate DPD in specific month using BALANCE_DT

In [7]:
query_parts = []
months = 24

for i in range(months + 1):
    target_month = i
    dpd_case = f"CASE"
    
    for j in range(1, months + 1):  # Loop through the 12 possible months
        dpd_case += f"""
                WHEN DATE_DIFF(balance_dt, DATE_SUB(DATE_TRUNC(DATE_ADD(open_dt, INTERVAL {target_month} MONTH), MONTH), INTERVAL 1 DAY), MONTH) = {j} THEN days_past_due_{str(j).zfill(2)}"""
    
    dpd_case += f"""
            END AS dpd_target_{target_month}m"""
    query_parts.append(dpd_case)

month_target_query = ",".join(query_parts)

### Use all the above sections to dynamically build a query

In [8]:
dt_updated_query = ',\n        '.join(updated_column_list)
tt_columns_query = ',\n            '.join(column_list)
dpd_query = ',\n        '.join(dpd_columns)

In [9]:
suit_filed_codes = [
    '1', 
    '01', 
    '2', 
    '02', 
    '3', 
    '03', 
    '200', 
    '201', 
    '202', 
    '203', 
    '204', 
    '205', 
    '206', 
    '207', 
    '208', 
    '209'
]

suit_filed_query = ', '.join([f"'{code}'" for code in suit_filed_codes])

written_off_settled_codes = [0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12]
written_off_settled_query = ', '.join([f"'{str(code).zfill(2)}'" for code in written_off_settled_codes])

In [10]:
prod_project = 'abcd-dataplatform'
app_db_dataset = 'abcd_data_science_app'
pl_leads_table = 'abcl_instapl_validation_base'
t_customer = 'ABCDPRODDB_t_customer'
sms_features = 'sms_features_data_temp'

### Get filtered Dataframe

In [11]:
QUERY = f"""/* **Query to for creating Alternate Credit Scoring Data**
Source Tables:
    1. t_customer:             abcd-dataplatform-prod.abcd_mobileapp_transformed.ABCDPRODDB_t_customer
    2. experian_daily_base:    abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED
    3. experian_adhoc_base:    abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_ADHOC_BASE_UPDATED (not used due to costing)
    
Process Outline:
    1. Get customers registered in a month from t_customer.
    2. Get the Latest data customer and loan account key level from Experian:
        a. Choose All rows with the latest Scrub Date.
        b. Select the latest reporting date (BALANCE_DT column).
    3. Apply general fixes:
        a. Parse all date columns to Python-readable format.
        b. If amount columns have values < 0, mark them as NaN.
        c. For OPEN_DT and CLOSED_DT columns, mark all values below their respective thresholds as NaT.
        d. If the DPD columns have -1, mark them as NaN.
        e. Fix NaN or missing values in the DPD columns where the adjacent columns give sensible values.
    4. Calculate specific month DPD for target mapping
*/

WITH customer_base AS (
-- CTE for Registered Customers in a month (Customer Base)

        SELECT DISTINCT
            CAST(mobilenumber AS STRING) AS mobilenumber,
            DATE(disbursal_date) AS disbursal_date
        FROM
            `{prod_project}.{app_db_dataset}.{pl_leads_table}`
),

latest_data AS (
-- CTE for Getting Latest Experian data update on Customer and Loan Account Level
-- Mark DPD column as NaN if it has values -1 or 900. Keep originally NaN values untouched.

    SELECT
        RANK() OVER (PARTITION BY exp.mobilenumber, ACCT_KEY ORDER BY SCRUB_DATE DESC, source ASC, balance_dt DESC) AS rank,
        {dt_updated_query},
        suit_filed_flag,
        written_off_settled_flag
    FROM (
        SELECT
            {tt_columns_query},
            MAX(CASE 
                    WHEN SUIT_FILED_WILLFUL_DFLT IN ({suit_filed_query})
                    THEN 1 
                    ELSE 0
                END) 
            OVER (PARTITION BY CUSTOMER_ID) AS suit_filed_flag,
            MAX(CASE 
                    WHEN WRITTEN_OFF_AND_SETTLED_STATUS IN ({written_off_settled_query})
                    THEN 1 
                    ELSE 0
                END) 
            OVER (PARTITION BY CUSTOMER_ID) AS written_off_settled_flag,
            1 AS source
        FROM
            `abffsl-dataplatform-uat.abfssl_central_analytics.EXPERIAN_RPT_AR_DAILY_BASE_UPDATED`
        WHERE
            CUSTOMER_ID IN (SELECT mobilenumber FROM customer_base)
            AND ACCT_TYPE_CD IN ('123')
    ) AS exp
    WHERE
        orig_loan_am > 25000
        AND open_dt >= '1900-01-01'
        AND open_dt <= CURRENT_DATE()
        AND m_sub_id IN ('ADITYA', 'Aditya_Birla', 'ABFL')
    QUALIFY rank = 1
),

quality_data AS (
-- Apply fixes for OPEN_DT, CLOSED_DT, ORIG_LOAN_AM, BALANCE_AM columns

    SELECT
        mobilenumber,
        ACCT_KEY,
        ACCT_TYPE_CD,
        {dpd_query},
        CASE 
            WHEN open_dt < DATE('1900-01-01') THEN NULL
            ELSE open_dt
        END AS open_dt,
        balance_dt,
        CASE 
            WHEN closed_dt < DATE('1899-12-30') THEN NULL
            ELSE closed_dt
        END AS closed_dt,
        credit_limit_am, 
        CASE
            WHEN orig_loan_am < 0 THEN NULL
            ELSE orig_loan_am
        END AS orig_loan_am,
        CASE
            WHEN balance_am < 0 THEN NULL
            ELSE balance_am
        END AS balance_am,
        suit_filed_flag,
        written_off_settled_flag,
        CASE
            WHEN suit_filed_flag = 1 OR written_off_settled_flag = 1
            THEN 1
            ELSE 0
        END AS charged_off_flag,
        scrub_date
    FROM
        latest_data
),


nan_fixed_data AS (
/* Apply fixes to the DPD columns and join with Customer Base

Fix DPD column when:
    1. Value of DPD column is NULL or <= 0
    2. DPD n-1 and DPD n+1 are NOT NULL
    3. Value(DPD n+1) - Value(DPD n-1) > 55
Fix to apply on DPD column:
    1. Add 30 to DPD n-1
    
Take the Customer Base and left join the fixed Bureau Data on Mobile Number 
*/

    SELECT
        cb.*,
        acct_key,
        acct_type_cd,
        open_dt,
        closed_dt,
        balance_dt,
        credit_limit_am,
        orig_loan_am,
        balance_am,
        suit_filed_flag,
        written_off_settled_flag,
        charged_off_flag,
        CASE
            WHEN exp.mobilenumber IS NULL THEN 1
            ELSE 0
        END AS ntc_flag,
        {dpd_nan_fix},
        {dpd_columns[-1]}
    FROM
        customer_base AS cb
    INNER JOIN
        quality_data AS exp
    ON
        cb.mobilenumber = exp.mobilenumber
        AND cb.disbursal_date = exp.open_dt
),

calculated_dpd AS (
/* **Final Query to get the Raw Data for Alternate Credit Scoring**

From the NaN fixed data, calculate the DPD for specific month by traversing backwards
from reporting date and check if:
    1. The target Month is reachable from reporting date
    2. If reachable, calculate the DPD for the target month as difference between
       target DPD column and the preceeeding DPD column
    3. If target DPD column is 0, mark target DPD as 0
    
For calculating DPD for specific month:
    1. Get the last date of the month before the specific month.
    2. With the difference, get the corresponding DPD column (DPD n)
    3. Report (DPD n) - (DPD n-1) as DPD for the month
*/

    SELECT
        *,{month_target_query},
        CASE 
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 1 THEN 'days_past_due_01'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 2 THEN 'days_past_due_02'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 3 THEN 'days_past_due_03'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 4 THEN 'days_past_due_04'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 5 THEN 'days_past_due_05'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 6 THEN 'days_past_due_06'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 7 THEN 'days_past_due_07'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 8 THEN 'days_past_due_08'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 9 THEN 'days_past_due_09'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 10 THEN 'days_past_due_10'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 11 THEN 'days_past_due_11'
            WHEN DATE_DIFF(balance_dt, disbursal_date, MONTH) = 12 THEN 'days_past_due_12'
            ELSE 'no_capture'
        END AS dpd_current_date_column    
    FROM
        nan_fixed_data
),

exclusion_customers AS (
    SELECT
        mobilenumber,
        MAX(dpd_target_0m) AS max_dpd_value
    FROM
        calculated_dpd
    GROUP BY
        mobilenumber
    HAVING
        MAX(dpd_target_0m) != 0
)

SELECT
    *,
    DATE_TRUNC(open_dt, MONTH) AS snapshot_partition,
    CASE
        WHEN dpd_current_date_column = 'no_capture' THEN 1
        ELSE 0
    END AS no_capture_flag,
    CASE
        WHEN mobilenumber IN (SELECT mobilenumber FROM exclusion_customers) THEN 1
        ELSE 0
    END AS current_dpd_customers_flag
FROM
    calculated_dpd
WHERE
    mobilenumber NOT IN (SELECT mobilenumber FROM exclusion_customers)
    AND dpd_current_date_column != 'no_capture'
    AND charged_off_flag = 0
ORDER BY
    mobilenumber,
    acct_key,
    balance_dt DESC
"""

In [12]:
alt_cs_qj_filtered = client.query(QUERY)
alt_cs_filtered = alt_cs_qj_filtered.to_dataframe()
alt_cs_filtered.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7699 entries, 0 to 7698
Data columns (total 68 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   mobilenumber                7699 non-null   object 
 1   disbursal_date              7699 non-null   dbdate 
 2   acct_key                    7699 non-null   object 
 3   acct_type_cd                7699 non-null   object 
 4   open_dt                     7699 non-null   dbdate 
 5   closed_dt                   89 non-null     dbdate 
 6   balance_dt                  7699 non-null   dbdate 
 7   credit_limit_am             7699 non-null   float64
 8   orig_loan_am                7699 non-null   float64
 9   balance_am                  7699 non-null   float64
 10  suit_filed_flag             7699 non-null   Int64  
 11  written_off_settled_flag    7699 non-null   Int64  
 12  charged_off_flag            7699 non-null   Int64  
 13  ntc_flag                    7699 

In [13]:
alt_cs_filtered.mobilenumber.nunique()

7644

In [14]:
alt_cs_filtered.head()

,mobilenumber,disbursal_date,acct_key,acct_type_cd,open_dt,closed_dt,balance_dt,credit_limit_am,orig_loan_am,balance_am,...,dpd_target_19m,dpd_target_20m,dpd_target_21m,dpd_target_22m,dpd_target_23m,dpd_target_24m,dpd_current_date_column,snapshot_partition,no_capture_flag,current_dpd_customers_flag
0,6000372862,2025-04-13,7192010133,123,2025-04-13,NaT,2025-08-15,169331.0,169331.0,154666.0,...,NaN,NaN,NaN,NaN,NaN,NaN,days_past_due_04,2025-04-01,0,0
1,6000503317,2025-04-30,7192034157,123,2025-04-30,NaT,2025-08-15,117000.0,117000.0,109946.0,...,NaN,NaN,NaN,NaN,NaN,NaN,days_past_due_04,2025-04-01,0,0
2,6000696628,2024-08-30,6392200812,123,2024-08-30,NaT,2025-08-15,100000.0,100000.0,78739.0,...,NaN,NaN,NaN,NaN,NaN,NaN,days_past_due_12,2024-08-01,0,0
3,6000834550,2024-09-10,6479745396,123,2024-09-10,NaT,2025-08-15,315000.0,315000.0,284471.0,...,NaN,NaN,NaN,NaN,NaN,NaN,days_past_due_11,2024-09-01,0,0
4,6001851260,2025-05-05,7218615570,123,2025-05-05,NaT,2025-08-15,130375.0,130375.0,122403.0,...,NaN,NaN,NaN,NaN,NaN,NaN,days_past_due_03,2025-05-01,0,0


### Get Unfiltered Dataframe

In [15]:
# Create 3m and 6m columns list
cols_2m = [f'dpd_target_{i}m' for i in range(1, 3)]
cols_3m = [f'dpd_target_{i}m' for i in range(1, 4)]
cols_6m = [f'dpd_target_{i}m' for i in range(1, 7)]
cols_1m = [f'dpd_target_{i}m' for i in range(1, 1)]
cols_ever = [f'dpd_target_{i}m' for i in range(1, months + 1)]

In [16]:
def filter_customers(df, customer_id_col, cols, check_value=None, check_null=False):
    """
    Filter out customers where all specified DPD columns have value 900 across all their rows.
    Uses NumPy for optimized performance.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    customer_id_col (str): Name of the customer ID column
    cols_3m (list): List of DPD column names to check
    
    Returns:
    pandas.DataFrame: Filtered DataFrame excluding customers with all 900s
    """
    # Convert relevant columns to NumPy array for faster computation
    dpd_array = df[cols].to_numpy()
    
    if check_null:
        # Create a mask for rows where all columns are null
        mask = (np.isnan(dpd_array).all(axis=1))
    else:
        # Create a mask for rows where all columns are 900
        mask = (dpd_array == check_value).all(axis=1)
    
    # Group by customer ID and check if all rows are 900
    customers_masked = df.groupby(customer_id_col)[cols].apply(
        lambda x: np.all(mask[x.index])
    )
    
    # Get customers to keep (those who don't have all 900s)
    customers_to_keep = customers_masked[~customers_masked].index
    
    # Filter the original dataframe
    return customers_masked[customers_masked].index.unique().tolist()

all_null_custs_6m = filter_customers(alt_cs_filtered, 'mobilenumber', cols_6m, check_null=True)
all_900_custs_6m = filter_customers(alt_cs_filtered, 'mobilenumber', cols_6m, check_value=900)
all_null_custs_3m = filter_customers(alt_cs_filtered, 'mobilenumber', cols_3m, check_null=True)
all_900_custs_3m = filter_customers(alt_cs_filtered, 'mobilenumber', cols_3m, check_value=900)
all_null_custs_2m = filter_customers(alt_cs_filtered, 'mobilenumber', cols_2m, check_null=True)
all_900_custs_2m = filter_customers(alt_cs_filtered, 'mobilenumber', cols_2m, check_value=900)
all_null_custs_1m = filter_customers(alt_cs_filtered, 'mobilenumber', ['dpd_target_1m'], check_null=True)
all_900_custs_1m = filter_customers(alt_cs_filtered, 'mobilenumber', ['dpd_target_1m'], check_value=900)

### Prepare for Target

In [17]:
targetting = alt_cs_filtered.copy()
targetting.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7699 entries, 0 to 7698
Data columns (total 68 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   mobilenumber                7699 non-null   object 
 1   disbursal_date              7699 non-null   dbdate 
 2   acct_key                    7699 non-null   object 
 3   acct_type_cd                7699 non-null   object 
 4   open_dt                     7699 non-null   dbdate 
 5   closed_dt                   89 non-null     dbdate 
 6   balance_dt                  7699 non-null   dbdate 
 7   credit_limit_am             7699 non-null   float64
 8   orig_loan_am                7699 non-null   float64
 9   balance_am                  7699 non-null   float64
 10  suit_filed_flag             7699 non-null   Int64  
 11  written_off_settled_flag    7699 non-null   Int64  
 12  charged_off_flag            7699 non-null   Int64  
 13  ntc_flag                    7699 

In [18]:
# Create Mask for 900 values
non_900_mask_6m = targetting[cols_6m] != 900
non_900_mask_3m = targetting[cols_3m] != 900
non_900_mask_2m = targetting[cols_2m] != 900
non_900_mask_1m = targetting['dpd_target_1m'] != 900
non_900_mask_ever = targetting[cols_ever] != 900

In [19]:
## 3 months 30 days
condition_3m_ge30 = (
    (targetting[cols_3m][non_900_mask_3m] >= 30).any(axis=1, skipna=True)
)

condition_3m_lt30 = (
    (targetting[cols_3m][non_900_mask_3m] < 60).all(axis=1, skipna=True)
    & (targetting[cols_3m] == 900).any(axis=1)
)

## 6 months 90 days
condition_3m_ge60 = (
    (targetting[cols_3m][non_900_mask_3m] >= 60).any(axis=1, skipna=True)
)

condition_3m_lt60 = (
    (targetting[cols_3m][non_900_mask_3m] < 60).all(axis=1, skipna=True)
    & (targetting[cols_3m] == 900).any(axis=1)
)

In [20]:
## 6 months 60 days
condition_6m_ge60 = (
    (targetting[cols_6m][non_900_mask_6m] >= 60).any(axis=1, skipna=True)
)

condition_6m_lt60 = (
    (targetting[cols_6m][non_900_mask_6m] < 60).all(axis=1, skipna=True)
    & (targetting[cols_6m] == 900).any(axis=1)
)

## 6 months 90 days
condition_6m_ge90 = (
    (targetting[cols_6m][non_900_mask_6m] >= 90).any(axis=1, skipna=True)
)

condition_6m_lt90 = (
    (targetting[cols_6m][non_900_mask_6m] < 90).all(axis=1, skipna=True)
    & (targetting[cols_6m] == 900).any(axis=1)
)

## Till Date 0+ days
condition_ever_ge90 = (
    (targetting[cols_ever][non_900_mask_ever] > 0).any(axis=1, skipna=True)
)

condition_ever_lt90 = (
    (targetting[cols_ever][non_900_mask_ever] <= 0).all(axis=1, skipna=True)
    & (targetting[cols_ever] == 900).any(axis=1)
)

## Till Date 30 days
condition_ever_ge30 = (
    (targetting[cols_ever][non_900_mask_ever] >= 30).any(axis=1, skipna=True)
)

condition_ever_lt30 = (
    (targetting[cols_ever][non_900_mask_ever] < 30).all(axis=1, skipna=True)
    & (targetting[cols_ever] == 900).any(axis=1)
)

In [21]:
targetting['target_3m_30days'] = np.where(
    condition_3m_ge30, 2,
    np.where(condition_3m_lt30, 1, 0)
)

targetting['target_3m_60days'] = np.where(
    condition_3m_ge60, 2,
    np.where(condition_3m_lt60, 1, 0)
)

targetting['target_6m_60days'] = np.where(
    condition_6m_ge60, 2,
    np.where(condition_6m_lt60, 1, 0)
)

targetting['target_6m_90days'] = np.where(
    condition_6m_ge90, 2,
    np.where(condition_6m_lt90, 1, 0)
)

targetting['target_12m_0+days'] = np.where(
    condition_ever_ge90, 2,
    np.where(condition_ever_lt90, 1, 0)
)

targetting['target_12m_30days'] = np.where(
    condition_ever_ge30, 2,
    np.where(condition_ever_lt30, 1, 0)
)

### Get Targets

1. 3 months window
    1. 30+
    2. 60+
2. 6 months window
    1. 60+
    2. 90+

In [22]:
targetting = targetting[targetting.dpd_current_date_column != 'no_capture']
targetting_agg = targetting.groupby(['mobilenumber', 'snapshot_partition']).agg(
    target_3m_30days=('target_3m_30days', 'max'),
    target_3m_60days=('target_3m_60days', 'max'),
    target_6m_60days=('target_6m_60days', 'max'),
    # target_6m_90days=('target_6m_90days', 'max'),
    target_12m_90days=('target_12m_0+days', 'max'),
    target_12m_30days=('target_12m_30days', 'max')
)

In [23]:
disb_amount = targetting.groupby(['mobilenumber', 'snapshot_partition']).agg(
    total_disb_amount=('orig_loan_am', 'sum')
)

In [24]:
disb_amount.reset_index().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7683 entries, 0 to 7682
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mobilenumber        7683 non-null   object 
 1   snapshot_partition  7683 non-null   dbdate 
 2   total_disb_amount   7683 non-null   float64
dtypes: dbdate(1), float64(1), object(1)
memory usage: 180.2+ KB


In [25]:
melted = targetting_agg.reset_index().melt(id_vars=['mobilenumber', 'snapshot_partition'], var_name='target', value_name='target_value')

melted['target_period'] = melted['target'].str.extract(r'target_(\d+m)')[0].replace({'3m': '3 months', '6m': '6 months', '12m': 'till date'})
melted['target_threshold'] = melted['target'].str.extract(r'(\d+days)')[0].replace({'30days': '30 days', '60days': '60 days', '90days': '0+ days'})

final = melted[['mobilenumber', 'snapshot_partition', 'target_period', 'target_threshold', 'target_value']]

final.loc[:, 'target_value'] = final['target_value'].replace({1: 2, 2: 1})

In [26]:
final = pd.merge(
    final,
    disb_amount.reset_index()[['mobilenumber', 'snapshot_partition', 'total_disb_amount']],
    on=['mobilenumber', 'snapshot_partition'],
    how='left'
)

In [27]:
final[['target_period']].value_counts(dropna=False).sort_index()

target_period
3 months         15366
6 months          7683
till date        15366
Name: count, dtype: int64

In [28]:
final = final[
    ~(
        (final.mobilenumber.isin(all_900_custs_3m) | final.mobilenumber.isin(all_null_custs_3m))
        & (final.target_period == '3 months')
    )
]

In [29]:
final = final[
    ~(
        (final.mobilenumber.isin(all_900_custs_6m) | final.mobilenumber.isin(all_null_custs_6m))
        & (final.target_period == '6 months')
    )
]

In [30]:
final.mobilenumber.nunique() - final[final.target_period == '3 months'].mobilenumber.nunique()

0

In [31]:
final.mobilenumber.nunique() - final[final.target_period == '6 months'].mobilenumber.nunique()

0

In [32]:
round(
    final.groupby(['target_period', 'target_threshold']).target_value.value_counts(dropna=False, normalize=True) * 100,
    2
)

target_period  target_threshold  target_value
3 months       30 days           0               99.51
                                 1                0.49
               60 days           0               99.91
                                 1                0.09
6 months       60 days           0               98.85
                                 1                1.15
till date      0+ days           0               90.43
                                 1                9.57
               30 days           0               95.29
                                 1                4.71
Name: proportion, dtype: float64

In [50]:
final[
    final.target_period == '3 months'
]

,mobilenumber,snapshot_partition,target_period,target_threshold,target_value,total_disb_amount
0,6000237962,2024-08-01,3 months,30 days,0,100000.0
1,6000376267,2024-08-01,3 months,30 days,0,85000.0
2,6000696856,2024-11-01,3 months,30 days,0,300000.0
3,6000702227,2025-04-01,3 months,30 days,0,142000.0
4,6000955927,2024-07-01,3 months,30 days,0,113514.0
...,...,...,...,...,...,...
147661,9999965363,2024-10-01,3 months,60 days,0,400000.0
147662,9999974318,2024-08-01,3 months,60 days,0,56000.0
147663,9999985698,2024-12-01,3 months,60 days,0,200000.0
147664,9999986956,2024-07-01,3 months,60 days,0,50000.0


In [33]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38415 entries, 0 to 38414
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mobilenumber        38415 non-null  object 
 1   snapshot_partition  38415 non-null  dbdate 
 2   target_period       38415 non-null  object 
 3   target_threshold    38415 non-null  object 
 4   target_value        38415 non-null  int64  
 5   total_disb_amount   38415 non-null  float64
dtypes: dbdate(1), float64(1), int64(1), object(3)
memory usage: 1.8+ MB


In [34]:
final = final.reset_index(drop=True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38415 entries, 0 to 38414
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   mobilenumber        38415 non-null  object 
 1   snapshot_partition  38415 non-null  dbdate 
 2   target_period       38415 non-null  object 
 3   target_threshold    38415 non-null  object 
 4   target_value        38415 non-null  int64  
 5   total_disb_amount   38415 non-null  float64
dtypes: dbdate(1), float64(1), int64(1), object(3)
memory usage: 1.8+ MB


In [35]:
# Define dataset and table ID
dataset_id = 'abcd_data_science_app'
table_id = f'abcd-dataplatform.{dataset_id}.risk_model_abcl_base'

# Sample schema
schema = [
    bigquery.SchemaField('mobilenumber', 'STRING'),
    bigquery.SchemaField('snapshot_partition', 'DATE'),
    bigquery.SchemaField('target_period', 'STRING'),
    bigquery.SchemaField('target_threshold', 'STRING'),
    bigquery.SchemaField('target_value', 'INTEGER'),
    bigquery.SchemaField('total_disb_amount', 'FLOAT'),
]

# Define the table with partitioning
table = bigquery.Table(table_id, schema=schema)

try:
    client.delete_table(table_id)
except Exception:
    pass

# Create the table
table = client.create_table(table, exists_ok=True)  # Will not fail if table already exists
print(f"Created partitioned table: {table.project}.{table.dataset_id}.{table.table_id}")

Created partitioned table: abcd-dataplatform.abcd_data_science_app.risk_model_abcl_base


In [53]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 369013 entries, 0 to 369164
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   mobilenumber        369013 non-null  object 
 1   snapshot_partition  369013 non-null  dbdate 
 2   target_period       369013 non-null  object 
 3   target_threshold    369013 non-null  object 
 4   target_value        369013 non-null  int64  
 5   total_disb_amount   369013 non-null  float64
dtypes: dbdate(1), float64(1), int64(1), object(3)
memory usage: 19.7+ MB


In [54]:
pandas_gbq.to_gbq(
    dataframe=final,
    destination_table=f'{dataset_id}.risk_model_abcl_base',
    project_id='abcd-dataplatform',
    if_exists='append'
)

100%|██████████| 1/1 [00:00<00:00, 7345.54it/s]


In [36]:
targets_backup = targetting_agg.copy(deep=True).reset_index()

In [37]:
targets_backup = targets_backup.rename(columns={
    'target_3m_30days': '3MOB30',
    'target_3m_60days': '3MOB60',
    'target_6m_60days': '6MOB60',
    'target_12m_90days': 'ever0',
    'target_12m_30days': 'ever30',
})

targets_backup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7683 entries, 0 to 7682
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mobilenumber        7683 non-null   object
 1   snapshot_partition  7683 non-null   dbdate
 2   3MOB30              7683 non-null   int64 
 3   3MOB60              7683 non-null   int64 
 4   6MOB60              7683 non-null   int64 
 5   ever0               7683 non-null   int64 
 6   ever30              7683 non-null   int64 
dtypes: dbdate(1), int64(5), object(1)
memory usage: 420.3+ KB


In [38]:
pandas_gbq.to_gbq(
    dataframe=targets_backup,
    destination_table=f'{dataset_id}.risk_model_abcl_instapl_base_unpivot',
    project_id='abcd-dataplatform',
    if_exists='replace'
)

100%|██████████| 1/1 [00:00<00:00, 11814.94it/s]


In [ ]:
%%bigquery

CREATE OR REPLACE TABLE `abcd-dataplatform.abcd_data_science_app.ensemble_model_jfm_validation_feature_base`
AS (
    SELECT
        mobilenumber,
        snapshot_partition
    FROM
        `abcd-dataplatform.abcd_data_science_app.ensemble_model_jfm_validation_onus_targets`
    
    UNION DISTINCT 

    SELECT
        mobilenumber,
        snapshot_partition
    FROM
        `abcd-dataplatform.abcd_data_science_app.ensemble_model_jfm_validation_offus_targets`
)

In [ ]:
final.head()

In [ ]:
final.columns

# Waterfall Analysis

In [ ]:
alt_cs.journey_flag = np.where(
    (alt_cs.acct_type_cd.isin(['123']))
    & (alt_cs.orig_loan_am > 25000)
    & (alt_cs.open_dt >= alt_cs.journey_start_date),
    1, 0
)

In [ ]:
customer_flags = alt_cs.groupby('customer_id').agg({
    'customer_id': 'size',
    'ntc_flag': 'sum',
    'no_capture_flag': 'sum',
    'current_dpd_customers_flag': 'sum',
    'charged_off_flag': 'sum',
    'apr_dec_snapshots_flag': 'sum',
    'journey_flag': 'max',
    'pl_flag' : 'max',
    'disb_amount_flag': 'max'
}).rename(columns={'customer_id': 'counts'}).reset_index()
registered_cust_count = customer_flags.shape[0]
registered_cust_count

In [ ]:
ntc_cust_count = customer_flags[
    (customer_flags.ntc_flag != 0)
].shape[0]
ntc_cust_count

In [ ]:
non_pl_count = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 0)
].shape[0]
non_pl_count

In [ ]:
low_disb_amount_count = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 0)
].shape[0]
low_disb_amount_count

In [ ]:
customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
].journey_flag.value_counts(dropna=False)

In [ ]:
early_journey_counts = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 0)
].shape[0]
early_journey_counts

In [ ]:
no_capture_cust_count = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts == customer_flags.no_capture_flag)
].shape[0]
no_capture_cust_count

In [ ]:
current_dpd_cust_count = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag != 0)
].shape[0]
current_dpd_cust_count

In [ ]:
derogatory_cust_count = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag != 0)
].shape[0]
derogatory_cust_count

In [ ]:
apr_dec_custs = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag == 0)
    & (customer_flags.apr_dec_snapshots_flag == 0)
].shape[0]
apr_dec_custs

In [ ]:
filtered_cust_counts = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag == 0)
    & (customer_flags.apr_dec_snapshots_flag != 0)
].shape[0]
filtered_cust_counts

In [ ]:
alt_cs_filtered.customer_id.nunique()

In [ ]:
registered_cust_count - ntc_cust_count - non_pl_count - low_disb_amount_count - early_journey_counts - no_capture_cust_count - current_dpd_cust_count - derogatory_cust_count - apr_dec_custs

In [ ]:
all_null_dpd_targets_cust_counts = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag == 0)
    & (customer_flags.apr_dec_snapshots_flag != 0)
    & (customer_flags.customer_id.isin(all_null_custs_3m))
].shape[0]
all_null_dpd_targets_cust_counts

In [ ]:
all_900_dpd_targets_cust_counts = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag == 0)
    & (customer_flags.apr_dec_snapshots_flag != 0)
    & (customer_flags.customer_id.isin(all_900_custs_3m))
].shape[0]
all_900_dpd_targets_cust_counts

In [ ]:
junk_dpd_counts = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag == 0)
    & (customer_flags.apr_dec_snapshots_flag != 0)
    & (customer_flags.customer_id.isin(all_null_custs_3m) | customer_flags.customer_id.isin(all_900_custs_3m))
].shape[0]
junk_dpd_counts

In [ ]:
final_cust_counts = customer_flags[
    (customer_flags.ntc_flag == 0)
    & (customer_flags.pl_flag == 1)
    & (customer_flags.disb_amount_flag == 1)
    & (customer_flags.journey_flag == 1)
    & (customer_flags.counts > customer_flags.no_capture_flag)
    & (customer_flags.current_dpd_customers_flag == 0)
    & (customer_flags.charged_off_flag == 0)
    & (customer_flags.apr_dec_snapshots_flag != 0)
    & ~(customer_flags.customer_id.isin(all_null_custs_3m) | customer_flags.customer_id.isin(all_900_custs_3m))
].shape[0]
final_cust_counts

In [ ]:
final[final.target_period == '3 months'].customer_id.nunique()

In [ ]:
waterfall_dict = {
    'Total Customers with PL Journey': registered_cust_count,
    'NTC customers': ntc_cust_count,
    'Customers with DPD not mapped': no_capture_cust_count,
    'Customers with Current DPD': current_dpd_cust_count,
    'Derogatory Flagged Customers': derogatory_cust_count,
    'Customers with no Disbursal in Apr-Dec': apr_dec_custs
}

pd.DataFrame(waterfall_dict, index=pd.Index([0]))